In [11]:
import os
import numpy as np
from tqdm import tqdm
import leafmap
from localtileserver import TileClient, get_leaflet_tile_layer
import rasterio as rio
import pandas as pd
import geopandas as gpd
from dem_stitcher.geojson_io import read_geojson_gzip
import matplotlib.pyplot as plt

In [12]:
# Set a workdir and go to it
workdir = '/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/oliver/rtc_visualization'
print('Going to work dir: ', workdir)
os.chdir(workdir)

Going to work dir:  /Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/oliver/rtc_visualization


In [13]:
# Load the CSV file into a DataFrame using Pandas
df_rtc = pd.read_json('/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/marshak/4_rtc_organization/rtc_s1_table.json.zip')
df_rtc


,rtc_s1_id,input_slc_id,jpl_burst_id,bursts_per_slc_input,rtc_s1_vv_url,rtc_s1_vh_url,rtc_s1_h5_url,acq_datetime
0,OPERA_L2_RTC-S1_T005-008688-IW2_20201010T00400...,S1A_IW_SLC__1SDV_20201010T004001_20201010T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-10 00:40:01
1,OPERA_L2_RTC-S1_T005-008688-IW2_20201022T00400...,S1A_IW_SLC__1SDV_20201022T004000_20201022T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-22 00:40:00
2,OPERA_L2_RTC-S1_T005-008688-IW2_20201103T00400...,S1A_IW_SLC__1SDV_20201103T004000_20201103T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-03 00:40:00
3,OPERA_L2_RTC-S1_T005-008688-IW2_20201115T00400...,S1A_IW_SLC__1SDV_20201115T004000_20201115T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-15 00:40:00
4,OPERA_L2_RTC-S1_T005-008688-IW2_20201127T00400...,S1A_IW_SLC__1SDV_20201127T004000_20201127T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-27 00:40:00
...,...,...,...,...,...,...,...,...
237848,OPERA_L2_RTC-S1_T175-374047-IW1_20211103T16352...,S1B_IW_SLC__1SDV_20211103T163459_20211103T1635...,T175-374047-IW1,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2021-11-03 16:34:59
237849,OPERA_L2_RTC-S1_T175-374047-IW1_20211115T16352...,S1B_IW_SLC__1SDV_20211115T163459_20211115T1635...,T175-374047-IW1,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2021-11-15 16:34:59
237850,OPERA_L2_RTC-S1_T175-374047-IW1_20211127T16352...,S1B_IW_SLC__1SDV_20211127T163459_20211127T1635...,T175-374047-IW1,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2021-11-27 16:34:59
237851,OPERA_L2_RTC-S1_T175-374047-IW1_20211209T16352...,S1B_IW_SLC__1SDV_20211209T163458_20211209T1635...,T175-374047-IW1,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2021-12-09 16:34:58


In [14]:
# filtered_df = rtc_table[rtc_table['jpl_burst_id'].str.startswith('T036-076225-IW1')]
# rtc_s1_vv_url_list = filtered_df['rtc_s1_vv_url'].tolist()
# filtered_count = len(filtered_df)
# print('Image count =', filtered_count)
# print('Sample link =', rtc_s1_vv_url_list[0])
# filtered_df

In [15]:
## Note: this method is slow scroll down for a better way 

# Read a group of images and display them using leafmap
# Create a LeafMap instance
# m = leafmap.Map()

# # Add the images to the map
# for rtc_path in rtc_s1_vv_url_list[0:5]:
#     # Add the image to the map
#     m.add_raster(rtc_path, colormap='gray', vmin=0, vmax=0.5)

# # Display the map
# m


#### Get a pixel TS

In [16]:
# ## Note: this method is a bit slow
# def load_image(image_path):
#     with rio.open(image_path) as src:
#         return src.read(1)

# def extract_center_pixel(image):
#     height, width = image.shape
#     center_row = height // 2
#     center_col = width // 2
#     return image[center_row, center_col]

# def plot_center_pixel_time_series(image_paths):
#     time_series = []

#     for image_path in image_paths:
#         image = load_image(image_path)
#         center_pixel_value = extract_center_pixel(image)
#         center_pixel_value = 10 * np.log10(center_pixel_value)
#         time_series.append(center_pixel_value)

#     plt.figure(figsize=(14, 7))
#     plt.plot(range(1, len(time_series) + 1), time_series, marker='o')
#     plt.title('Center Pixel Time Series')
#     plt.xlabel('Image #')
#     plt.ylabel('Pixel Value dB')
#     plt.grid(True)
#     plt.show()

# # List of image paths
# image_paths = [rtc_s1_vv_url_list[1], rtc_s1_vv_url_list[2], rtc_s1_vv_url_list[3], rtc_s1_vv_url_list[4]]
# #rtc_s1_vv_url_list

# # Plot center pixel time series
# plot_center_pixel_time_series(rtc_s1_vv_url_list)

#### Burst query and plot TS

In [17]:
BURST_ID = 'T036-076225-IW1'
df_burst_ts = df_rtc[df_rtc.jpl_burst_id == BURST_ID].reset_index(drop=True)
df_burst_ts.head()
print('Total dates for burst', BURST_ID, '=', len(df_burst_ts))

Total dates for burst T036-076225-IW1 = 100


In [18]:
# Read burst summary table generated in dist-s1-research/marshak/4_rtc_organization/0_Organize-RTC-Data.ipynb
df_burst_summary = read_geojson_gzip('/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/marshak/4_rtc_organization/burst_extent_subset.geojson.zip')
# from the burst summary extract the specific burst info (we will use the geometry to define center of plot)
df_burst = df_burst_summary[df_burst_summary.jpl_burst_id == BURST_ID].reset_index(drop=True)
df_burst

,geometry,jpl_burst_id,orbit_pass,count,is_val_burst,track_number
0,"POLYGON ((21.47216 4.92982, 20.71055 5.09328, ...",T036-076225-IW1,DESCENDING,100,False,36


In [19]:
# Create stack
layers = {d: leafmap.common.get_local_tile_layer(url, vmin=0, vmax=.25) for (d, url) in zip(tqdm(df_burst_ts.acq_datetime[:]), df_burst_ts.rtc_s1_vh_url)}

100%|██████████| 100/100 [00:35<00:00,  2.80it/s]


In [20]:
centroid = df_burst.geometry[0].centroid
m = leafmap.ts_inspector(layers, center=(centroid.y, centroid.x), zoom=11)
# if validation sites available, includem with the line below
# m.add_gdf(df_val_sites, 
      # layer_name=f"Val. Sites", info_mode='on_click')
m

Map(center=[4.912536882828104, 21.068059136628108], controls=(AttributionControl(options=['position', 'prefix'…

#### Create tiles and display

In [21]:
# Layer rasters
J = 0 # image number to tile (first)
src_1 = rio.open(df_burst_ts.rtc_s1_vh_url.tolist()[J])
client_1 = TileClient(src_1)

K = -1 # image number to tile (last)
src_2 = rio.open(df_burst_ts.rtc_s1_vh_url.tolist()[K])
client_2 = TileClient(src_2)


In [22]:
m = leafmap.Map(center=(centroid.y, centroid.x))
m.add_basemap("Esri.WorldImagery")

m.add_raster(client_1, vmin=0, vmax=.25, layer_name=df_burst_ts.acq_datetime.tolist()[J])
K = -1
m.add_raster(client_2, vmin=0, vmax=.25, layer_name=df_burst_ts.acq_datetime.tolist()[K])
m

Map(center=[4.912536882828104, 21.068059136628108], controls=(ZoomControl(options=['position', 'zoom_in_text',…

#### Create a side by side display

In [23]:
# Get a vh and vv to display 
N = 0
src_vv = rio.open(df_burst_ts.rtc_s1_vv_url.tolist()[N])
client_vv = TileClient(src_vv)

src_vh = rio.open(df_burst_ts.rtc_s1_vh_url.tolist()[N])
client_vh = TileClient(src_vh)

In [24]:
# Split pane display

from ipyleaflet import Map, ScaleControl, FullScreenControl, SplitMapControl

m = leafmap.Map(center=(centroid.y, centroid.x), zoom=12)

# Shared display parameters
display = dict(vmin=0, vmax=.25, colormap='binary_r')

# Create 2 tile layers from different raster
l = get_leaflet_tile_layer(client_vv, **display)
r = get_leaflet_tile_layer(client_vh, **display)


control = SplitMapControl(left_layer=l, right_layer=r)
m.add_control(control)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl())
m

Map(center=[4.912536882828104, 21.068059136628108], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [25]:
# # Read rtc information table generated in dist-s1-research/marshak/4_rtc_organization/0_Organize-RTC-Data.ipynb
# df_rtc = pd.read_json('/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/marshak/4_rtc_organization/rtc_s1_table.json.zip')
# df_rtc.head()